In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from Lookup_ViT import LookupViT

In [ ]:
model = LookupViT(
    in_channels=1,
    dim=64,
    heads=4,
    depth=6,
    num_classes=10,
    image_size=128,
    lookup_patch_size=16
)

In [ ]:
print(model)

In [ ]:
# 导入minist
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 配置参数
batch_size = 64  # 每个批次的样本数
shuffle = True   # 是否在每个 epoch 后打乱数据

# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),  # 转换为 PyTorch 张量
    transforms.Normalize((0.5,), (0.5,)),# 标准化到 [-1, 1]
    transforms.Resize((128, 128))  # 调整图像大小
])

# 加载 MNIST 数据集
train_dataset = datasets.MNIST(
    root='./data',         # 数据存储路径
    train=True,            # 加载训练集
    transform=transform,   # 数据预处理
    download=True          # 如果数据不存在，下载数据
)

test_dataset = datasets.MNIST(
    root='./data',
    train=False, 
    transform=transform,
    download=True
)

# 创建数据加载器
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=shuffle
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)



In [ ]:
epochs = 50
lr = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)


In [ ]:

def train_epoch(model, data_loader, criterion, optimizer, device):
    
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    
    for data, target in data_loader:
        data = data.to(device)
        target = target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()
        
    return train_loss / len(data_loader), correct / total

In [ ]:
def test_epoch(model, data_loader, criterion, device):
    
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data, target in data_loader:
            data = data.to(device)
            target = target.to(device)
            
            output = model(data)
            loss = criterion(output, target)
            
            test_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
            
    return test_loss / len(data_loader), correct / total

In [ ]:
#for epoch in range(epochs):
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = test_epoch(model, test_loader, criterion, device)
    
    print(f'Epoch [{epoch}/{epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}')